# Importando Libs


In [ ]:
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer, util

# Analisando Df

In [5]:
celulares_df = pd.read_csv("/home/caua/GoogleDrive/ENGENHARIA/COMPUTAÇÃO/MATÉRIAS SEMESTRE 2/PROJETOS 2/AMBIENTE/dataset_celulares_final.csv")
display(celulares_df.head())


,Nome,Marca,Ano,Preco_Num,RAM_Num,Specs_Completa,Armazenamento_Num,Bateria_Num,Url
0,Samsung Galaxy Z TriFold,Samsung,01/12/2025,0.00,16.0,Smartphone: Samsung Galaxy Z TriFold. Marca: S...,1.0,5600.0,https://www.oficinadanet.com.br/smartphones/sa...
1,Samsung Galaxy A17 4G,Samsung,08/09/2025,1198.00,8.0,Smartphone: Samsung Galaxy A17 4G. Marca: Sams...,256.0,5000.0,https://www.oficinadanet.com.br/smartphones/sa...
2,Samsung Galaxy S25 FE,Samsung,04/09/2025,3098.00,8.0,Smartphone: Samsung Galaxy S25 FE. Marca: Sams...,512.0,4900.0,https://www.oficinadanet.com.br/smartphones/sa...
3,Samsung Galaxy A07 4G,Samsung,23/08/2025,646.94,8.0,Smartphone: Samsung Galaxy A07 4G. Marca: Sams...,256.0,5000.0,https://www.oficinadanet.com.br/smartphones/sa...
4,Samsung Galaxy A17 5G,Samsung,06/08/2025,1249.00,8.0,Smartphone: Samsung Galaxy A17 5G. Marca: Sams...,256.0,5000.0,https://www.oficinadanet.com.br/smartphones/sa...


In [6]:
celulares_df.iloc[0,5]

'Smartphone: Samsung Galaxy Z TriFold. Marca: Samsung. Lançamento: 01/12/2025. Status: Lançamento. Data lançamento: 01/12/2025. Outras informações: Dimensões. Dimensões: Aberto: 159.2 x 214.1 x 3.9-4.2 mm e Fechado: 159.2 x 75 x 12.9 mm. Peso: 309g. Construção: Vidro Frontal (Gorilla Glass Victus Ceramic 2) Tela externa e Frame de alumínio. Chip: Multi eSIM e Nano-SIM. Proteção Água: IP48. Dobrável: Sim. Plataforma: Sistema. Sistema: Android 16 e One UI 8. Atualizações: 7 anos (sistema). Processador: Qualcomm Snapdragon 8 Elite (3 nm). CPU: Octa-core (2x4.47 GHz Oryon V2 Phoenix L, 6x3.53 GHz Oryon V2 Phoenix M). GPU: Adreno 830. Memórias: Memória RAM. Memória RAM: 16 GB. Tipo de Memória RAM: UFS 4.0. Armazenamento: 512 GBe1 TB. Armazenamento Extra: Não Possui. Tela: Tela - Tipo. Tela - Tipo: Tri-foldable Dynamic LTPO AMOLED 2X. Tela - Tamanho: 10 polegadas. Tela - Resolução: 1584 x 2160 pixels. Tela - Frequência: 120Hz. Tela - Densidade: 269 ppi. Tela - Proteção: Corning Gorilla Glass

In [4]:
contagem = celulares_df["Marca"].value_counts()
display(contagem)

Marca
Xiaomi      156
Motorola    123
Realme      117
Samsung     108
Honor        54
Oppo         51
Apple        27
Infinix      27
Asus         12
Name: count, dtype: int64

In [5]:
celulares_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 675 entries, 0 to 674
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Nome               675 non-null    object 
 1   Marca              675 non-null    object 
 2   Ano                675 non-null    object 
 3   Preco_Num          675 non-null    float64
 4   RAM_Num            675 non-null    float64
 5   Specs_Completa     675 non-null    object 
 6   Armazenamento_Num  675 non-null    float64
 7   Bateria_Num        675 non-null    float64
 8   Url                675 non-null    object 
dtypes: float64(4), object(5)
memory usage: 47.6+ KB


# Recomendado modelo

## Simulando com diferentes entradas

In [30]:
# --- SIMULAÇÃO 1: GAMER OSTENTAÇÃO ---

query = "Smartphone gamer, processador Snapdragon Elite ou superior, GPU potente Adreno, tela com alta taxa de atualização 120Hz ou 144Hz, resfriamento avançado. Tela dobrável, Foldable, Tri-fold, tecnologia flexível. alto armazenamento interno. Bateria de longa duração 5000mAh, autonomia para um dia, eficiência energética. Suporte a rede 5G. Tecnologia NFC. Wireless charging, carregamento por indução."

user_choices = {
    "orcamento_max": 99999.00,       # Opção "Sem limites"
    "marcas_preferidas": ["Samsung"], # Preferência por Samsung
    "target_ram": 8,                 # Veio do perfil "Gamer"
    "target_armazenamento": 256,     # Veio de "O Acumulador"
    "target_bateria": 5000,          # Veio de "Bateria Tranquila"
}

In [34]:
# --- SIMULAÇÃO 2: TRABALHADOR CUSTO-BENEFÍCIO ---

query = "Multitarefa eficiente, muita memória RAM, modo desktop DeX, tela grande para leitura, processamento rápido. Tela grande, display imersivo para mídia. Bateria gigante de longa duração 6000mAh ou 5000mAh, autonomia para dois dias, eficiência energética. Suporte longo de sistema, 7 anos de atualizações, garantia de update Android."

user_choices = {
    "orcamento_max": 3500.00,        # Opção "Até R$ 3.500"
    "marcas_preferidas": [],         # Lista vazia = "Tanto faz"
    "target_ram": 12,                # Veio do perfil "Trabalho & Produtividade" (Filtro agressivo!)
    "target_armazenamento": 0,       # Veio de "Sou básico" (Sem filtro)
    "target_bateria": 6000,          # Veio de "Bateria Gigantesca para 2 dias"
}

In [38]:
# --- SIMULAÇÃO 3: FOTÓGRAFO INFLUENCER ---

query = "Câmera de alta resolução 200MP ou 108MP, estabilização ótica OIS, zoom ótico, ultrawide, fotos noturnas, gravação 4K. Tela tamanho padrão confortável. alto armazenamento interno. Bateria de longa duração 5000mAh, autonomia para um dia, eficiência energética. Proteção IP68, IP48, resistência a água, Gorilla Glass Victus, construção robusta."

user_choices = {
    "orcamento_max": 6000.00,                  # Opção "Até R$ 6.000"
    "marcas_preferidas": ["Apple", "Samsung"], # Marcou as duas gigantes
    "target_ram": 0,                           # Fotografia não exigiu RAM mínima no questionário
    "target_armazenamento": 256,               # Veio de "O Acumulador" (Crucial para vídeo)
    "target_bateria": 5000,                    # Veio de "Bateria Tranquila"
}

In [42]:
# --- SIMULAÇÃO 4: BÁSICO DESASTRADO ---

query = "Celular com bom custo benefício, fluido para redes sociais, tela de boa qualidade. Tela tamanho padrão confortável. Bateria de longa duração 5000mAh, autonomia para um dia, eficiência energética. Proteção IP68, IP48, resistência a água, Gorilla Glass Victus, construção robusta."

user_choices = {
    "orcamento_max": 1200.00,                    # Opção "Até R$ 1.200"
    "marcas_preferidas": ["Motorola", "Xiaomi"], # Marcas custo-benefício
    "target_ram": 0,                             # Uso básico não tem filtro de RAM
    "target_armazenamento": 0,                   # Uso básico não tem filtro de memória
    "target_bateria": 5000,                      # Veio de "Bateria Tranquila"
}

In [50]:
# --- SIMULAÇÃO 5: MONSTRO DA BATERIA ---

query = "Bateria gigante de longa duração 6000mAh ou 5000mAh, autonomia para dois dias, eficiência energética. Tela grande, display imersivo para mídia. Bateria gigante de longa duração 6000mAh ou 5000mAh, autonomia para dois dias, eficiência energética. Suporte a rede 5G."

# Nota: A frase de bateria aparece duplicada na query porque ele escolheu o PERFIL bateria E o TAMANHO bateria.
# Isso é BOM! Reforça o peso desse assunto para o Embedding.

user_choices = {
    "orcamento_max": 2000.00,                        # Opção "Até R$ 2.000"
    "marcas_preferidas": ["Xiaomi", "Realme", "Infinix"], 
    "target_ram": 0,
    "target_armazenamento": 0,
    "target_bateria": 6000,                          # Filtro rigoroso: Só aceita >= 6000mAh
}

## Simulando

In [51]:
# --- CÉLULA DE FILTRO (HARD FILTERS) ---

# 1. Criar uma cópia para preservar o original
banco_filtravel = celulares_df.copy()

# === NOVA LINHA: LIMPEZA DE DADOS ===
# Remove aparelhos com preço zerado ou negativo
banco_filtravel = banco_filtravel[banco_filtravel['Preco_Num'] > 0]

print(f"Total inicial de aparelhos (com preço válido): {len(banco_filtravel)}")

# 2. Filtro de Preço (Obrigatório)
if user_choices.get("orcamento_max", 0) > 0:
    banco_filtravel = banco_filtravel[banco_filtravel['Preco_Num'] <= user_choices['orcamento_max']]

# 3. Filtro de Marca (Opcional)
if user_choices.get("marcas_preferidas"):
    banco_filtravel = banco_filtravel[banco_filtravel['Marca'].isin(user_choices['marcas_preferidas'])]

# 4. Filtro de RAM (Requisito Mínimo)
if user_choices.get("target_ram", 0) > 0:
    banco_filtravel = banco_filtravel[banco_filtravel['RAM_Num'] >= user_choices['target_ram']]

# 5. Filtro de Armazenamento (Requisito Mínimo)
if user_choices.get("target_armazenamento", 0) > 0:
    banco_filtravel = banco_filtravel[banco_filtravel['Armazenamento_Num'] >= user_choices['target_armazenamento']]

# 6. Filtro de Bateria (Requisito Mínimo)
if user_choices.get("target_bateria", 0) > 0:
    banco_filtravel = banco_filtravel[banco_filtravel['Bateria_Num'] >= user_choices['target_bateria']]

# Resetar o index para organizar a tabela final
banco_filtravel = banco_filtravel.reset_index(drop=True)

# --- RESULTADO DO FUNIL ---
print(f"Total final após filtros: {len(banco_filtravel)}")
display(banco_filtravel[['Nome', 'Preco_Num', 'Marca', 'RAM_Num', 'Armazenamento_Num']].head())

Total inicial de aparelhos (com preço válido): 312
Total final após filtros: 6


,Nome,Preco_Num,Marca,RAM_Num,Armazenamento_Num
0,Xiaomi POCO X7 Pro,1399.99,Xiaomi,12.0,512.0
1,Xiaomi Redmi Note 14 Pro,1779.00,Xiaomi,16.0,512.0
2,Xiaomi POCO X7 Pro,1399.99,Xiaomi,12.0,512.0
3,Xiaomi Redmi Note 14 Pro,1779.00,Xiaomi,16.0,512.0
4,Xiaomi POCO X7 Pro,1399.99,Xiaomi,12.0,512.0


In [52]:

# --- PEÇA 1: O MODELO (O TRADUTOR) ---
model = SentenceTransformer('all-MiniLM-L6-v2') 

# Aqui a IA trabalha. Ela transforma texto em vetor.
vetor_banco = model.encode(banco_filtravel['Specs_Completa']) 
vetor_usuario = model.encode(query)

# --- PEÇA 2: O ALGORITMO (O MATEMÁTICO) ---
# Aqui NÃO TEM IA. É pura álgebra linear (multiplicação de matrizes).
# 'util.cos_sim' é a função que calcula o ângulo.
scores = util.cos_sim(vetor_usuario, vetor_banco)

In [53]:
indice_vencedor = torch.argmax(scores).item()
display(banco_filtravel.iloc[indice_vencedor,:])


Nome                                                Xiaomi POCO X7 Pro
Marca                                                           Xiaomi
Ano                                                         09/01/2025
Preco_Num                                                      1399.99
RAM_Num                                                           12.0
Specs_Completa       Smartphone: Xiaomi POCO X7 Pro. Marca: Xiaomi....
Armazenamento_Num                                                512.0
Bateria_Num                                                     6000.0
Url                  https://www.oficinadanet.com.br/smartphones/xi...
Name: 0, dtype: object

Lembre de ver valores que tem memoria de 1t como 1, eles não serão recomendados, termine isso ainda hj